# Working with Pandas

Pandas has dataframes (df) and df are intuitively readable tables and that is very useful!

In the notebook we will create Pandas DFs, learn methods to search the content, read CSV tables and apply simple transformations to the data. If you want to learn more about Pandas, have a look [here](https://www.w3schools.com/python/pandas/default.asp).

The first step is to import Pandas. It should already be installed in Anaconda. If not, contact us or try a code cell with the following: `%pip install pandas` (only needs to be run once).

In [ ]:
#pip install pandas

In [ ]:
import pandas as pd

## Dataframe

We create the first df “manually”, i.e. we first create a dictionary and then transfer it to Pandas.

Using `print()` on a df does not look very good. We instead use `display()`.

In [ ]:
#Dictionary
data = {"name": ["Otto", "Oskar", "Othmar"],
        "groesse": [181.3, 115.3, 153.4], 
        "geburtstag": ["1983-07-18", "2019-09-21", "2017-03-01"],
        }

#Dictionary to Pandas
fam = pd.DataFrame(data=data)

display(fam)

What column names are in the df?

In [ ]:
fam.columns

What information can we find in the “name” column?

In [ ]:
fam["name"]

What data type does the “namen” column have?

In [ ]:
type(fam["name"])

Oops, that looks complicated. But we can also convert the Pandas series into a simple list:

In [ ]:
fam["name"].to_list()

What is in the first (i.e. 0th) line of the data frame?

In [ ]:
fam.iloc[0]

What is in the first two (0, 1) lines of the data frame?

In [ ]:
fam.iloc[:2]

What is in the first two rows of the first column? (Python always indexes the rows first and then the columns)

In [ ]:
fam.iloc[:2, 0]

## CSV tables

We don't actually want to create dfs ourselves, but rather read them from existing CSV tables, for example. This is what we do here. You have to choose the right `Seperator` and also the corresponding `Encoding` (character system -> otherwise the German Umlaute are wrong)

By the way, the CSV is from [here](https://opendata.swiss/de/dataset/nachnamen-der-standigen-wohnbevolkerung-nach-kanton-1). It contains the frequency of surnames per canton. Physically, the CSV is in the same folder as the notebook (so we don't need to specify a path, just the name of the CSV).

### Read CSV

In [ ]:
nachnamen = pd.read_csv("nachnamen.csv", sep=",", encoding="utf-8")

display(nachnamen.head(5))

`info()` gives you a quick overview of all columns and their data types

In [ ]:
print(nachnamen.info())

Data type `object` means that Python is either not sure (e.g. mixed data types in a column) or that the column is of type string.

### Write CSV

Because it was so easy to read, we write the Pandas table straight back into a CSV. We only select a few columns (using double brackets: `[[]]`) and the first 10 rows (`.head(10)`).

`index=False` prevents Python from adding a first column that contains row numbers.

In [ ]:
nachnamen[["TIME_PERIOD", "LASTNAME", "GDEKT"]].head(10).to_csv("nachnamen_small.csv", index=False)

By the way, if individual lines of code get a bit long and are no longer clear, you can put them in brackets and then insert line breaks:

In [ ]:
(
    nachnamen[["TIME_PERIOD", "LASTNAME", "GDEKT"]]
    .head(10)
    .to_csv("nachnamen_small.csv", index=False)
)

## Loop in dataframes

Looping through the rows of a df is a bit special. With `iterrows()` we get two outputs, the row index (`idx`) on the one hand and the current row as a list (`row`) on the other.

Ah, and by the way, we use `If-Else` here to read only the first rows. We could perhaps do this more simply (e.g. `head(10)` for the dataframe), but this way you have also seen how to interrupt a loop using `break`.

In [ ]:
for index, row in nachnamen.iterrows():
    if index<10:
        print(row['TIME_PERIOD'], row['LASTNAME'])
    else:
        break

## Filtering

A typical first step to clean the data is filtering.

### Filtering by a single value

We pass a series of booleans to the dataframe. Wherever the canton is *ZH*, the booleans are `True`. Consequently, only the *ZH* rows are retained.

In [ ]:
nachnamen_zh = nachnamen[nachnamen["GDEKT"] == "ZH"]

display(nachnamen_zh.head(5))

In [ ]:
nachnamen_haeufig = nachnamen[nachnamen["VALUE"] > 5000]

display(nachnamen_haeufig.head(5))

### Filter by multiple values in the same column

The `isin()` function allows us to pass a list of possible canton abbreviations.

In [ ]:
nachnamen_zh_ag = nachnamen[nachnamen["GDEKT"].isin(["ZH", "AG"])]

display(nachnamen_zh_ag.head(5))

### Filter by multiple values in different columns

- Multiple statements must be wrapped individually in brackets
- The vertical bar (`|`) means *or* to make an *and* intersection between two statements you can use `&`
- Here we use a function from the `str` class as the second statement, namely `startswith()`, to obtain only surnames beginning with *M*.

In [ ]:
nachnamen_haeufig = nachnamen[(nachnamen["VALUE"] > 5000) | (nachnamen["LASTNAME"].str.startswith("M"))]

display(nachnamen_haeufig.head(5))

## Missing Data

Another typical task: Search, remove, repalce missing data.

First we need some data with missing information (`None`)

In [ ]:
data = {'A': [1, 2, None, 4, None], 
        'B': [None, 2, 3, 4, None]
        }

missing = pd.DataFrame(data)

display(missing)

Count missing data per column (`axis=1`)

In [ ]:
missing.isna().sum(axis=0)

Filter rows with any missing values

In [ ]:
missing.dropna()

Filter rows where a specific column has missing values

In [ ]:
missing[missing['A'].notna()]

Remove rows where all elements are missing

In [ ]:
missing.dropna(how='all')

Replace NaN with a specific value

In [ ]:
missing.fillna(0)

Replace NaN in a specific column

In [ ]:
missing['A'].fillna(value=999)

Replace NaN using forward fill (propagate last valid value forward). There is also a backward-fill: `bfill()`

In [ ]:
missing.ffill()

## Aggregate data frames

Aggregating information is a fundamental task when processing and analyzing data. Here I show just a few examples. The topic is almost endless.

### Counting

How many rows does the DF have?

In [ ]:
shape = nachnamen.shape
length = len(nachnamen)

print(f"Der Dataframe hat {shape[0]} (oder {length}) Zeilen")

### Sum

What is the sum of all surname frequencies? (shouldn't that roughly correspond to the population of Switzerland?)

In [ ]:
nachnamen["VALUE"].sum()

### Grouping

Grouping or `groupBy` is a comprehensive topic. We are just touching on it here. If you would like to know more, you can find out more [here](https://realpython.com/pandas-groupby/).

- We group by last name: `groupby(['LASTNAME'])`
- Then we aggregate: 
    - Frequency of each surname (sum of canton frequencies): `lastname_sum=('VALUE', 'sum')`
    - Lowest canton frequencies: `lastname_min=('VALUE', 'min')`
    - Number of cantons in which each surname occurs: `lastname_n_cantons=('GDEKT', 'nunique')`
- Then we sort the whole thing so that the most frequent surnames come first (`sort_values(“lastname_sum”, ascending=False)`) and we reset the index (for whatever reason!?)

There are different `groupBy` syntaxes. I particularly like this one (see especially `agg()`):

In [ ]:
nachnamen_grp = (
    nachnamen
    .groupby(['LASTNAME'])
    .agg(
        lastname_sum=('VALUE', 'sum'),
        lastname_min=('VALUE', 'min'),
        lastname_n_kantone=('GDEKT', 'nunique')
    )
    .sort_values("lastname_sum", ascending=False)
    .reset_index()
)

nachnamen_grp.head(5)

### Remove duplicates

We often find duplicates in the data. If you want to remove duplicates that affect all columns (real duplicates), the following will help:

In [ ]:
nachnamen.drop_duplicates(inplace = True)

If duplicates are to be removed in relation to individual columns only, you can use the `subset` parameter. `keep=“first”` means that the first column is kept for each series of duplicates. You can check which is the first column by prefixing it with `sort`.

Here, for example, we only keep the most frequent surname per canton per year (remove canton duplicates but sort by frequency first):

In [ ]:
nachnamen_haeufigst = (nachnamen
                    .sort_values(by='VALUE', ascending=False) # ascending=False heisst absteigend
                    .drop_duplicates(subset=['GDEKT', 'TIME_PERIOD'], keep='first')
                    )

nachnamen_haeufigst.head(5)

### Long2Wide and Wide2Long Transformations

Moving information from columns to rows or from rows to columns, is certainly not the first thing that comes to mind. 
Unfortunately, it happens quite often in programming. Statistical models, for example, like to store information in columns (i.e. each row is a unique sample), while visualization is easier if the information is stored in rows.

We make up some data to demonstrate Long2Wide first, using `pivot()`:

In [ ]:
data = {
    'Date': ['2024-11-01', '2024-11-01', '2024-11-02', '2024-11-02', '2024-11-03'],
    'Product': ['A', 'B', 'A', 'B', 'A'],
    'Sales': [100, 150, 200, 250, 300]
}
prod = pd.DataFrame(data)

display(prod)

We would like to have a table where the Date is in the rows (`index`) and the two Products are in two seperate columns (`columns`).

In [ ]:
prod_w = prod.pivot(index=['Date'], columns='Product', values='Sales').reset_index()

display(prod_w)

We can undo this, i.e. Wide2Long transformatino, with `melt()`.

In [ ]:
prod_l = (
    pd.melt(
        prod_w, 
        id_vars=["Date"], 
        var_name='Product', 
        value_name='Sales')
)

display(prod_l)

## Working with date & time

Working with date and time fields is somewhat annoying. We use the manually created table with the three birthdays:

In [ ]:
print(fam.info())

You can see that birthday has been saved as *object* (actually *str*). To be able to work with it, we need to reformat the birthday column into a date format. For this we use the Pandas function `pd.to_datetime()`.

In [ ]:
fam["geburtstag"] = pd.to_datetime(fam["geburtstag"])

print(fam.info())

Now we can calculate with the date. For example, we can calculate how much older Othmar is than Oskar (we do this with `iloc[]`, which is always a bit confusing):

In [ ]:
diff = fam.iloc[2, 2] - fam.iloc[1, 2]
diff

The difference between two date fields is returned in `Timedelta()` format. This in turn can be converted into a more common format:

In [ ]:
diff.days

## Combine dataframes

Finally to Pandas, but very important, the merging of different tables. 

### Join
For our example, we can consider, for example, specifying the frequency of surnames as percentages of the total population of the cantons instead of absolute numbers.

To do this, we need a second table with the resident population per canton. We could download this from OpenData.ch and import it. As there are only a few rows, I create the table manually.

The information on the population of the canton must now be added to each line of the surname table. This is usually called a *join*.

In [ ]:
data = {
    'Kanton': ['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 
                     'FR', 'SO', 'BS', 'BL', 'SH', 'AR', 'AI', 'SG', 'GR', 
                     'AG', 'TG', 'TI', 'VD', 'VS', 'NE', 'GE', 'JU'],
    'Bevoelkerung': [1538565, 1034977, 416347, 36707, 160480, 38576, 43160, 40653, 127387,
                   325822, 275596, 201971, 289534, 83041, 55630, 16145, 510734, 200288, 
                   700466, 282909, 353343, 814762, 345504, 176850, 504128, 73419]
}

# Create DataFrame
bev_kanton = pd.DataFrame(data)

- We take the table with the surnames (*left*) and add the population per canton (*right*)
- We use the `merge()` function (there would also be a `join()` function)
- In the surnames we have the column `GDEKT`, which matches the column `canton` in the population table: `left_on='GDEKT', right_on='Kanton'`
- We select an `inner` join. We only keep the rows that are contained in both tables (for example, if a canton in the name table does not appear in the population table, it is not included in the join).

A more comprehensive discussion of joins can be found [here](https://pandas.pydata.org/docs/user_guide/merging.html).

In [ ]:
nachnamen_bev = pd.merge(nachnamen, bev_kanton, left_on='GDEKT', right_on='Kanton', how='inner')

display(nachnamen_bev.head(5))

Now we just need to calculate the percentages in a new field. Optionally, we can sort the dataframe to see which surnames are the most common:

In [ ]:
nachnamen_bev["VALUE_REL"] = nachnamen_bev["VALUE"] / nachnamen_bev["Bevoelkerung"]

nachnamen_bev.sort_values("VALUE_REL", ascending=False).head(10)

Ah, a typical *small data problem*. Surnames in small cantons are, relatively speaking, the ones with the broadest distribution.

### Concatenate

A common form of merging data frames is the combination of two data sets with the same or similar information. Rows are added rather than columns as in a *join*. This is called *concatenate* (or *union*).

Here we take the table with the family members and combine it with a new, second table containing a few more people.

In [ ]:
data = {"name": ["Obama", "Taylor"], "groesse": [187.5, 178.1], "beruehmt": [True, True]}

#Dictionary to Pandas
fam_zusatz = pd.DataFrame(data=data)

display(fam_zusatz)

The new table does not contain all the columns of the family table (*birthday*), but has an additional column (*birthday*). The columns are also arranged differently. However, this does not matter for the *concatenate*.

- We use the `concat()` function of Pandas
- The columns are automatically arranged 'correctly' and missing information is added with `NaN` (*Not a Number*)

In [ ]:
fam_gross = pd.concat([fam, fam_zusatz], ignore_index=True)

display(fam_gross)